In [1]:
from general_tools.notebook import gpu_utils
GPU = 1
gpu_utils.setup_one_gpu(GPU)

Picking GPU 1


In [2]:
import sys
import time
import numpy as np
import os.path as osp
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.stats import hmean
from sklearn.manifold import TSNE

from general_tools.notebook.tf import reset_tf_graph

import tf_lab.point_clouds.in_out as pio
from tf_lab.point_clouds.in_out import PointCloudDataSet, write_model_ids_of_datasets
from tf_lab.point_clouds.point_net_ae import PointNetAutoEncoder
from tf_lab.point_clouds.autoencoder import Configuration as Conf
import tf_lab.point_clouds.encoders_decoders as enc_dec


from tf_lab.autopredictors.scripts.helper import shape_net_category_to_synth_id, points_extension, \
                                                 shape_net_core_synth_id_to_category


from tf_lab.autopredictors.plotting import plot_original_pclouds_vs_reconstructed, \
                                           plot_train_val_test_curves, plot_reconstructions_at_epoch
        
from tf_lab.autopredictors.evaluate import eval_model, read_saved_epochs, accuracy_of_completion, \
                                           coverage_of_completion, save_reconstructions, \
                                           save_pc_prediction_stats, save_stats_of_multi_class_experiments, \
                                           paper_pc_completion_experiment_id_best_epoch
                                                  
from tf_lab.autopredictors.exploration import latent_embedding_of_entire_dataset, \
                                              embedding_of_entire_dataset_at_tensor

from general_tools.in_out.basics import create_dir, delete_files_in_directory, files_in_subdirs
from general_tools.simpletons import select_first_last_and_k
from geo_tool import Point_Cloud

from tf_lab.nips.data_sets.model_net import pc_loader, classes_to_integers
from tf_lab.nips.helper import center_pclouds_in_unit_sphere, average_per_class, zero_mean_half_sphere

from tf_lab.point_clouds.encoders_decoders import decoder_with_fc_only
from tf_lab.point_clouds.encoders_decoders import encoder_with_convs_and_symmetry
from sklearn.svm import LinearSVC

/orions4-zfs/projects/lins2/Panos_Space/Git_Repos/geo_tool/solids/mesh.py:26: UserWarning: Mayavi library was not found. Some graphics utilities will be disabled.
  warnings.warn('Mayavi library was not found. Some graphics utilities will be disabled.')


In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [24]:
n_pc_samples = 2048
put_in_usphere = True

model_net = '40'

# experiment_name = 'wu_classes_rotated_mlp_arch_2048pts_chamfer'
experiment_name ='all_snc_rotated_conv_arch_2048pts_chamfer'

In [25]:
top_data_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/'
train_dir = osp.join(top_data_dir, 'OUT/models/nips/vanilla_ae/')
train_dir = osp.join(train_dir, experiment_name)

conf = Conf.load(osp.join(train_dir, 'configuration'))
print conf
conf.n_output = conf.n_input

              allow_gpu_growth: False
                    batch_size: 50
                 consistent_io: None
                         debug: False
                       decoder: decoder_with_fc_only
                  decoder_args: {'layer_sizes': [1024, 2048, 6144]}
                       encoder: encoder_with_convs_and_symmetry
                  encoder_args: {'filter_sizes': [40, 20, 10, 10], 'n_filters': [128, 128, 256, 512], 'strides': [1, 2, 2, 1]}
               experiment_name: all_snc_rotated_conv_arch_2048pts_chamfer
                 gauss_augment: None
                  is_denoising: False
               latent_vs_recon: 1.0
                 learning_rate: 0.0005
                          loss: chamfer
             loss_display_step: 1
                       n_input: [2048, 3]
                      n_output: [2048, 3]
                           n_z: None
             saver_max_to_keep: None
                    saver_step: 10
                     train_dir: /orions4-zfs/proj

In [26]:
reset_tf_graph()
ae = PointNetAutoEncoder(experiment_name, conf)
saved_epochs = read_saved_epochs(conf.train_dir)
last_epoch = saved_epochs[-1]
ae.restore_model(conf.train_dir, last_epoch, verbose=True)

Model restored in epoch 340.


In [27]:
model_net_dir = '/orions4-zfs/projects/lins2/Panos_Space/DATA/Point_Clouds/Model_Net_' + model_net +  '/from_manifold/' 
model_net_dir = osp.join(model_net_dir, str(n_pc_samples))

search_pattern = '(.*)train(.*)\.ply$'
train_pc_files = [f for f in files_in_subdirs(model_net_dir, search_pattern)]

search_pattern = '(.*)test(.*)\.ply$'
test_pc_files = [f for f in files_in_subdirs(model_net_dir, search_pattern)]

In [28]:
pc, model_names, labels = pio.load_crude_point_clouds(train_pc_files, loader=pc_loader, n_threads=10, verbose=True)

if put_in_usphere:
#     pc = center_pclouds_in_unit_sphere(pc)
    pc = zero_mean_half_sphere(pc)
    
train_data = PointCloudDataSet(pc, labels=labels)

pc, model_names, labels = pio.load_crude_point_clouds(test_pc_files, loader=pc_loader, n_threads=10, verbose=True)

if put_in_usphere:
#     pc = center_pclouds_in_unit_sphere(pc)
    pc = zero_mean_half_sphere(pc)
    
test_data = PointCloudDataSet(pc, labels=labels)

9841 pclouds were loaded. They belong in 40 shape-classes.
2467 pclouds were loaded. They belong in 40 shape-classes.


In [29]:
train_feed, train_latent, train_classes = latent_embedding_of_entire_dataset(train_data, ae, conf)
cids = classes_to_integers(int(model_net), train_classes)[1]

In [30]:
lsvc = LinearSVC(C=0.7, loss='squared_hinge', intercept_scaling=2.0, dual=False)
lsvc.fit(train_latent, cids)

LinearSVC(C=0.7, class_weight=None, dual=False, fit_intercept=True,
     intercept_scaling=2.0, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [31]:
test_feed, test_latent, test_classes = latent_embedding_of_entire_dataset(test_data, ae, conf)
cids_ = classes_to_integers(int(model_net), test_classes)[1]
print lsvc.score(test_latent, cids_)
print average_per_class(lsvc, test_latent, cids_)

0.86218078638
0.810570073439


In [44]:
[n.name for n in ae.graph.get_operations()]

code_tr_0 = embedding_of_entire_dataset_at_tensor(train_data, ae, conf, 'all_snc_2048pts_chamfer_1/decoder_fc_0/BiasAdd:0')[1]
code_te_0 = embedding_of_entire_dataset_at_tensor(test_data, ae, conf, 'all_snc_2048pts_chamfer_1/decoder_fc_0/BiasAdd:0')[1]


multi_feat_tr = np.hstack((code_tr_0, train_latent))
print multi_feat_tr.shape


(9841, 1536)
